In [1]:
import os
import torch
import argparse
import nbimporter
from tqdm.auto import tqdm
from transformers import GPT2LMHeadModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from dataloader import GPTDataLoader
from utils import generate

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name", default="skt/kogpt2-base-v2", type=str)
    parser.add_argument("--data_dir", default="C:/Users/NM333-68/LLM/KoGPT2/datasets/preprocessed", type=str)
    parser.add_argument("--batch_size", default=32, type=int)
    parser.add_argument("--epochs", default=10, type=int)
    parser.add_argument("--lr", default=2e-5, type=float)
    parser.add_argument("--warmup_steps", default=200, type=int)
    
    # Jupyter 노트북에서 인자 파싱을 위한 추가 코드
    import sys
    if 'ipykernel' in sys.modules:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()

    BASE_DIR = os.getcwd()
    DATA_DIR = os.path.join(BASE_DIR, args.data_dir)

    print(f"BASE_DIR: {BASE_DIR}")
    print(f"DATA_DIR: {DATA_DIR}")

    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    tokenizer.add_special_tokens({"pad_token": "<pad>"}) 

    train_dataloader = GPTDataLoader(
        tokenizer, os.path.join(DATA_DIR, "finetuning_data.csv"), args.batch_size
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = GPT2LMHeadModel.from_pretrained(args.model_name)
    model.resize_token_embeddings(len(tokenizer))
    model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=args.lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=len(train_dataloader) * args.epochs
    )

min_loss = float('inf')
for epoch in range(args.epochs):
    print(f"Training epoch {epoch}")
    total_loss = 0
    for input_text in tqdm(train_dataloader):
        input_tensor = input_text.to(device)
        outputs = model(input_tensor, labels=input_tensor)
        loss = outputs[0]
        total_loss += loss.item()
        optimizer.zero_grad()
        model.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"epoch {epoch} avg_loss {avg_loss:.2f}")

    categories = ["일상 대화", "주문", "요청", "건강", "교육"]
    for category in categories:
        gen = generate(category, tokenizer, model, device, max_length=50)
        print(f"{category} : {gen}")

    if avg_loss < min_loss:
        min_loss = avg_loss
        best_model_path = os.path.join(BASE_DIR, "best_model")
        model.save_pretrained(best_model_path)

print("학습 완료되었습니다.")

BASE_DIR: C:\Users\NM333-68\LLM\KoGPT2
DATA_DIR: C:/Users/NM333-68/LLM/KoGPT2/datasets/preprocessed


C:\Users\NM333-68\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading data from: C:/Users/NM333-68/LLM/KoGPT2/datasets/preprocessed\finetuning_data.csv
Data columns: Index(['id', 'Category', 'Text'], dtype='object')
Concatenated data sample: ['주문|애덜앙 나 너무 배불러서 배 아파', '주문|배 아프면 약국 가서 소화제라도 마시는 거 어때', '주문|얼마나 먹었다고 배가 아프데', '주문|떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어', '주문|하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지']
Dataset length: 1445943


C:\Users\NM333-68\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\NM333-68\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0


...

epoch 7 avg_loss 1.20
일상 대화 : ['우리 아빠도 좋아하시더라.']
주문 : ['어디서 받아?']
요청 : ['이제 그만 뒀어요.']
건강 : ['나도 그런 거 잘 몰라.']
교육 : ['학원에서 수업하면 되잖아']
Training epoch 8


  0%|          | 0/45186 [00:00<?, ?it/s]

epoch 8 avg_loss 1.19
일상 대화 : ['맞아요 그래서 내가 꼭 이기라고 했어요.']
주문 : ['내일 점심에 시켜먹어볼까?']
요청 : ['이번에 아르바이트로 과자 한박스 샀잖아 하하']
건강 : ['네 들어보긴 했지만 잘 기억이 안나요']
교육 : ['열심히 하고 싶어. 화이팅!']
Training epoch 9


  0%|          | 0/45186 [00:00<?, ?it/s]

epoch 9 avg_loss 1.18
일상 대화 : ['아니면 우리집에서 같이 자고 와!']
주문 : ['어플있지 거기']
요청 : ['오래 걸려야해서요']
건강 : ['그렇겠지? 나도 선생님들한테 한번 배워야 할 것 같아']
교육 : ['나도 토익이라 해 놓고 다음 날은 학원비 내면서 다 까먹었어.']
학습 완료되었습니다.
